In [ ]:
import asyncio
import websockets
import wave
import time
import nest_asyncio  # Fix Jupyter event loop issue

nest_asyncio.apply()  # Allow nested event loops

SERVER_URI = "ws://localhost:8000"
AUDIO_FILE_PATH = "C:/Users/Von3002/Desktop/Final_year_project/videoplayback.wav"

CHUNK = 1024  # Same as before
KEEP_ALIVE_INTERVAL = 10  # Send a keep-alive message every 10 seconds

async def send_audio():
    """Streams an uploaded audio file in an infinite loop without connection drops."""
    
    while True:
        try:
            async with websockets.connect(SERVER_URI, ping_interval=KEEP_ALIVE_INTERVAL) as websocket:
                print("🔊 Streaming audio file...")

                # Open the audio file
                with wave.open(AUDIO_FILE_PATH, 'rb') as audio:
                    last_keep_alive = time.time()

                    while True:  # Infinite loop for seamless playback
                        data = audio.readframes(CHUNK)
                        
                        if not data:  
                            audio.rewind()  # Restart from beginning before buffer exhausts
                            data = audio.readframes(CHUNK)

                        await websocket.send(data)  # Send audio data to server
                        
                        # Keep WebSocket alive
                        if time.time() - last_keep_alive > KEEP_ALIVE_INTERVAL:
                            await websocket.send(b"KEEP_ALIVE")  # Send keep-alive packet
                            last_keep_alive = time.time()

                        await asyncio.sleep(0.01)  # Small buffer delay to avoid bottlenecks

        except (websockets.exceptions.ConnectionClosedError, websockets.exceptions.ConnectionClosedOK) as e:
            print(f"🔄 Connection lost. Reconnecting in 3 seconds... ({e})")
            time.sleep(3)  # Prevent immediate reconnect spam

# Start streaming the file
asyncio.create_task(send_audio())  # Runs in the background